# Siamese network implementation with the strips dataset

In [1]:
import os
import numpy as np
import random as rng
import cv2
import json

In [2]:
info_data = []
with open('/home/data/strips_socrates/dataset_info.json') as json_file: 
    info_data = json.load(json_file) 

In [3]:
STRIP_SIZE = 256

In [4]:
def get_batch(info_data, batch_size, with_id=False, dataset = 'train'):
    pairs = [np.zeros((batch_size, STRIP_SIZE, STRIP_SIZE, 3)) for i in range(2)]
    labels = np.zeros((batch_size, ))
    labels[batch_size//2:] = 1
    split_index = int((len(info_data)-1) * 0.8)
    if dataset == 'train':
        students = [rng.randint(0, split_index) for _ in range(batch_size)] 
    elif dataset == 'whole':
        students = [rng.randint(0, (len(info_data)-1)) for _ in range(batch_size)] 
    else:
        students = [rng.randint(split_index+1, len(info_data)-1) for _ in range(batch_size)] 
    imgs = [rng.randint(0, len(info_data[i])-1) for i in students]
    strips_loc = [rng.randint(0, len(info_data[i][0][1])-1) for i in students]
    id_pairs = []
    for i in range(batch_size):
        std = students[i]
        img = imgs[i]
        srl = strips_loc[i]
        strip1 = info_data[std][img][1][srl]
        strip2 = ""
        if i >= batch_size // 2:
            img2 = (img + rng.randint(1, len(info_data[std])-1)) % len(info_data[std])
            strip2 = info_data[std][img2][1][srl]
        else:
            std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
            img2 = rng.randint(0, len(info_data[std2])-1)
            srl2 = rng.randint(0, len(info_data[std2][0][1])-1)
            strip2 = info_data[std2][img2][1][srl2]
        pairs[0][i,:,:,:] = cv2.imread(strip1)/255
        pairs[1][i,:,:,:] = cv2.imread(strip2)/255
        id_pairs.append((strip1, strip2))
    if with_id:
        return pairs, labels, id_pairs
    else:
        return pairs, labels

In [5]:
p,l,ids = get_batch(info_data, 10, True)

In [6]:
for x in zip(l, ids):
    print(x)

(0.0, ('/home/data/strips_socrates/111/Eurecom_111_picFG_007_256_1536.PNG', '/home/data/strips_socrates/126/Eurecom_126_picFG_004_2048_768.PNG'))
(0.0, ('/home/data/strips_socrates/132/Eurecom_132_picFG_017_0_2560.PNG', '/home/data/strips_socrates/123/Eurecom_123_picFG_002_4096_512.PNG'))
(0.0, ('/home/data/strips_socrates/140/Eurecom_140_picBG_030_768_1280.PNG', '/home/data/strips_socrates/113/Eurecom_113_picBG_005_2560_256.PNG'))
(0.0, ('/home/data/strips_socrates/110/Eurecom_110_picBG_032_3328_2304.PNG', '/home/data/strips_socrates/126/Eurecom_126_picBG_023_768_1280.PNG'))
(0.0, ('/home/data/strips_socrates/115/Eurecom_115_picBG_043_1280_1536.PNG', '/home/data/strips_socrates/139/Eurecom_139_picBG_043_256_512.PNG'))
(1.0, ('/home/data/strips_socrates/118/Eurecom_118_picBG_021_1024_0.PNG', '/home/data/strips_socrates/118/Eurecom_118_picFG_040_1024_0.PNG'))
(1.0, ('/home/data/strips_socrates/102/Eurecom_102_picBG_030_256_0.PNG', '/home/data/strips_socrates/102/Eurecom_102_picFG_039_25

In [7]:
def generate(info_data, batch_size, dataset='train'):
    while True:
        pairs, labels = get_batch(info_data, batch_size, False, dataset=dataset)
        yield(pairs, labels)

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [9]:
def sequential_block(input_shape = (256,256,3), base_filters=64):
    model = keras.Sequential()
    model.add(layers.Conv2D(base_filters, (4,4), input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(base_filters*4, (5,5)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(base_filters*4, (5,5)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(base_filters*4, (5,5)))
    model.add(layers.Dense(base_filters*32, activation='relu'))
    model.add(layers.Flatten())
    
    return model

In [ ]:
seq = sequential_block(base_filters=8)

In [ ]:
seq.summary()

In [10]:
def get_siamese_model(input_shape = (256,256,3), base_filters=16):


    left_input = layers.Input(input_shape)
    right_input = layers.Input(input_shape)
    
    # Convolutional Neural Network
    model = sequential_block(input_shape, base_filters)
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = layers.Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = layers.Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = keras.Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [15]:
model = get_siamese_model(base_filters=16)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 294912)       265232      input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 294912)       0           sequential_1[0][0]         

In [16]:
optimizer = keras.optimizers.Adam(lr = 1e-4)
model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['acc'])

In [17]:
batch_size = 64
epochs = 100
steps_epoch = 64

In [18]:
model.fit(generate(info_data, batch_size), 
         epochs = epochs,
         steps_per_epoch= steps_epoch,
         shuffle = True)

Epoch 1/100
64/64 [==============================] - 35s 482ms/step - loss: 0.8581 - acc: 0.5176
Epoch 2/100
64/64 [==============================] - 31s 486ms/step - loss: 0.7036 - acc: 0.5378
Epoch 3/100
64/64 [==============================] - 31s 483ms/step - loss: 0.7040 - acc: 0.5314
Epoch 4/100
64/64 [==============================] - 31s 484ms/step - loss: 0.7035 - acc: 0.5264
Epoch 5/100
64/64 [==============================] - 31s 485ms/step - loss: 0.6954 - acc: 0.5515
Epoch 6/100
64/64 [==============================] - 31s 485ms/step - loss: 0.7055 - acc: 0.5186
Epoch 7/100
64/64 [==============================] - 31s 485ms/step - loss: 0.6943 - acc: 0.5450
Epoch 8/100
64/64 [==============================] - 31s 485ms/step - loss: 0.6972 - acc: 0.5272
Epoch 9/100
64/64 [==============================] - 31s 486ms/step - loss: 0.7015 - acc: 0.5360
Epoch 10/100
64/64 [==============================] - 31s 486ms/step - loss: 0.6978 - acc: 0.5592
Epoch 11/100
64/64 [=========

### Validation:
Simple, just feed pairs from the validation split and see the performance.

### Extended validation:
Take two images, and compare strip by strip, then do a majority voting.